# Todo

## База
- Класс блока
- Класс линии
- Рисовальщик

## Алгоритм
- Добавление блока на линию
- Валидатор

In [1]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(request.items(), columns=['name', 'qty'])
df

,name,qty
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1


In [4]:
import sys
sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')

In [5]:
from app.models import *
from app.models import Boiling as BoilingModel

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [7]:
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())

In [8]:
df

,name,qty,sku
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>


In [9]:
sku = df.iloc[0]['sku']

In [10]:
session.query(BoilingModel).first()

<Boiling 1>

In [11]:
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())

In [12]:
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [ ]:
lines = ['cheese_maker', 'melting', 'termizator', 'packing']

In [67]:
import interval

def calc_interval_length(i):
    return sum([c[1] - c[0] for c in i])

In [200]:


PERIODS_PER_HOUR = 12
PERIODS_PER_DAY = 24 * PERIODS_PER_HOUR


class Block:
    def __init__(self, size=None, t=None, block_class=None, tags=None):
        self.size = size or 0
        self.t = t or 0
        
        self.children = []
        self.block_class = block_class
        self.tags = tags
        
    @property
    def beg(self):
        return self.t
    
    @property
    def end(self):
        return self.t + self.size
    
    @property
    def interval(self):
        return interval.interval[self.beg, self.end]
    
    def __str__(self):
        return '-'.join([str(self.beg), str(self.end)])
    
    def __repr__(self):
        return str(self)
    
    
    def push(self, block, after=None, before=PERIODS_PER_DAY):
        after = after or 0
        
        cur_t = after
        
        while cur_t < before:
            block.t = cur_t
            
            validated = self.validate(block)
            
            if validated:
                self.add(block)
                return True
            else:
                # try to fit the block
                cur_t += 1
    
    def add(self, block):
        self.children.append(block)
        
    def validate(self, block):
        for b in self.children:
            validated = validate(b, block)
            if not validated:
                return False
        return True

        
def validate(b1, b2):
    b1, b2 = min([b1, b2], key=lambda b: str(b)), max([b1, b2], key=lambda b: str(b))
    
    if calc_interval_length(b1.interval & b2.interval) != 0:
        return False
    
    return True


    
class WaterPouringBlock(Block):
    def __init__(self, name, periods, t=None):
        self.name = name 
        size = sum(periods.values()) - periods['pouring']
        super().__init__(size, t, block_class='water_pouring')
        
        self.blocks = {}
        self.blocks['pouring'] = Block(periods['pouring'], t=0)
        
        for key in ['pouring_and_fermenting', 'soldifictaion', 'cutting', 'pouring_off', 'extra']:
            self.blocks[key] = Block(periods[key], t=cur_t, tags=[key])
            cur_t += periods[key]



In [201]:
total_size = 27
water_pouring = Block(block_class='water_pouring', size=total_size)

upper_block = Block(size=total_size)
upper_block.push(Block(block_class='pouring', size=6, tags={'block_num': 12}))
upper_block.push(Block(block_class='pouring_name', size=total_size - 6, tags={'name': 'чильеджина  3,6 альче  8500кг'}))

lower_block = Block(size=total_size)
lower_block.push(Block(block_class='pouring_and_fermenting', size=8))
lower_block.push(Block(block_class='soldification', size=7))
lower_block.push(Block(block_class='cutting', size=7))
lower_block.push(Block(block_class='pouring_off', size=3))
lower_block.push(Block(block_class='extra', size=total_size - 8 - 8 - 7 - 3))

water_pouring.add(upper_block)
water_pouring.add(lower_block)

In [202]:
water_pouring

0-27

In [218]:
style = {
    'pouring': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
}

# Drawing

In [204]:
from matplotlib import colors

def cast_color(obj):
    if isinstance(obj, (tuple, list)):
        if all(isinstance(x, int) for x in obj):
            return '#{:02x}{:02x}{:02x}'.format(*obj)
        elif all(isinstance(x, float) for x in obj):
            return colors.to_hex(obj)
    elif isinstance(obj, str):
        try:
            return cast_color(colors.to_rgb(obj))
        except:
            pass
    raise Exception('Unknown object type')

In [205]:
def draw_block(x, y, w, h, text, colour):
    if not colour:
        colour = cast_color('white') # default white colour
    SHEET.merge_cells(start_row=y, start_column=x, end_row=y + h - 1, end_column=x + w - 1)
    merged_cell = SHEET.cell(row=y, column=x)  
    merged_cell.value = text
    merged_cell.alignment = Alignment(horizontal='center')
    merged_cell.fill = PatternFill("solid", fgColor=colour[1:])
    
def draw_block_series(x, y, h, blocks):
    cur_x = x
    for w, text, colour in blocks:
        draw_block(cur_x, y, w, h, text, colour)
        cur_x += w
        
def draw_two_line_series(x, y, h, blocks1, blocks2):
    assert sum([block[0] for block in blocks1]) == sum([block[0] for block in blocks2])
    draw_block_series(x, y, h, blocks1)
    draw_block_series(x, y + h, h, blocks2)
    
    
def cast_w(duration):
    # one width is five minutes
    return int(duration / 5)

In [206]:
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import openpyxl as opx
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
import json

In [191]:
# WB = opx.load_workbook(r'2020.10.06 pouring_template.xlsx')
# SHEET = WB.worksheets[0]

# draw_block(10, 10, 1, 1, '12 налив', '#008000')

# draw_block_series(8, 10, 1, 
#                   [
#                       [5, '60 налив', ''], 
#                       [27, 'фиор ди латте  3,3 альче 80 юнитов  8000кг/11,5кг галактаза 5200', '']
#                   ])

# draw_two_line_series(10, 11, 1,
#                   [
#                       [5, '60 налив', ''], 
#                       [27, 'фиор ди латте  3,3 альче 80 юнитов  8000кг/11,5кг галактаза 5200', '']
#                   ],
#                   [
#                       [12, 'налив/внесение закваски', '#FFA500'],
#                       [7, 'схватка', '#FFFF00'],
#                       [8, 'резка/обсушка', '#008000'],
#                       [3, 'слив', '#FF0000'],
#                       [2, '', '']
#                   ])

# draw_block(20, 20, cast_w(15), 1, 'a', '#008000')


WB.save('output.xlsx')

In [207]:
def draw(x, y, h, block):
    if block.children:
        for child in block.children:
            draw(x + child.t, y, h, child)
    else:
        tags = block.tags or {}
        tags.update(style.get(block.block_class, {}))
        text = tags.get('text', '')
        color = cast_color(tags.get('color', 'white'))
        draw_block(x, y, block.size, h, text.format(**tags), color)

In [215]:
WB = opx.load_workbook(r'2020.10.06 pouring_template.xlsx')
SHEET = WB.worksheets[0]

In [216]:
draw(10, 10, 1, water_pouring.children[0])
draw(10, 11, 1, water_pouring.children[1])

In [217]:
WB.save('output.xlsx')